# Case Study: House Price Prediction
---
**Problem statement:**
Real Estate company RealEstateML would like to study the house prices in Brookside, VT.

RealEstateML provided the following datasets to your team:
- house price
- school district
- population
- crime data

**Deliverable:**
Can you build a Linear Regression model using Python to predict the housing prices in Brookside, VT?
*(discovery questions)*


# Data Dictionary

house price: `house price.csv`

| column | description |
| ----------- | ----------- |
| House id | house id after anonymization |
| House price (k) | House price in US dollars, measured by 1,000 |
| size | House size in square feet |
| number of bedrooms | Number of bedrooms in this house |
| school district | School district this house belongs to |
| zip code | The zip code of the house location |

school district: `school_district.csv`

| column | description |
| ----------- | ----------- |
| school district | school district id, matches school district in 'house price' |
| school rating | Rating from “Great! Schools”, higher score are better |

population: `population.csv`

| column | description |
| ----------- | ----------- |
| zip | zip code |
| Population(k) | population measured by 1000 |

crime data: `crime_data.csv`

| column | description |
| ----------- | ----------- |
| zip code | zip code |
| Safety Score (0-10) | higher score indicates less crime in the zip code |



# Load the data

In [ ]:
# import the data (from CSV files on the web)
import pandas as pd

house_data = pd.read_csv(r'https://raw.githubusercontent.com/odonnell31/linear_regression_ML/main/house_price.csv',
                         dtype={'zip code': str})
school_data = pd.read_csv(r'https://raw.githubusercontent.com/odonnell31/linear_regression_ML/main/school_district.csv')
population_data = pd.read_csv(r'https://raw.githubusercontent.com/odonnell31/linear_regression_ML/main/population.csv',
                              dtype={'zip': str})
crime_data = pd.read_csv(r'https://raw.githubusercontent.com/odonnell31/linear_regression_ML/main/crime_data.csv',
                         dtype={'zip code': str})

In [ ]:
# check the shape of each dataframe
for dataset in [('house_data', house_data),
                ('school_data', school_data),
                ('population_data', population_data),
                ('crime_data', crime_data)]:
  print(f"{dataset[0]}")
  print(f"shape: {dataset[1].shape}")
  print(f"columns: {dataset[1].columns}")
  print("-"*5)

# Email from RealEstateML:
Please consider the followings before you start your analysis:
- The main goal is to create a prediction model that can estimate a house's price based on the variables we have about that house.
- To clarify a few points in the data:
  - the zip code 16199 data was accidentally deleted in the population data. The population is 132K.
  - S1010 is a simple typo for S10110
  - the safety score over 10 is error data and can be replaced by the average
  - you can handle missing value as you see fit
  - please remove duplicate records


# Data Preprocessing *(important)*

## Incorrect values

In [ ]:
# population_data: add zip code 16199 with population
population_data.loc[len(population_data.index)] = ['16199', 132]

In [ ]:
# school_data: fix school district 'S1010'
school_data['school district'].replace('S1010', 'S10110', inplace=True)

In [ ]:
# crime_data: set 'Safety Score (0-10)' > 10 to the mean value
mean_safety_score = crime_data['Safety Score (0-10)'].mean()
crime_data.loc[crime_data['Safety Score (0-10)'] > 10, 'Safety Score (0-10)'] = mean_safety_score

## How should we handle missing values?

In [ ]:
# handle missing values
for ds in [('house_data', house_data),
           ('school_data', school_data),
           ('population_data', population_data),
           ('crime_data', crime_data)]:
  print(f"dropping {ds[1].shape[0] - ds[1].dropna().shape[0]} rows from {ds[0]} due to missing values")
  ds[1].dropna(inplace=True)

## How should we remove duplicate records?

In [ ]:
# remove duplicate records
def remove_duplicates(df: pd.DataFrame,
                      df_name: str,
                      subset_columns: list) -> pd.DataFrame:

  num_duplicates = df.duplicated(subset=subset_columns).sum()
  print(f"dropping {num_duplicates} duplicate rows from {df_name}")
  df = df.drop_duplicates(subset=subset_columns)
  return df

In [ ]:
house_data = remove_duplicates(df = house_data,
                               df_name = "house_data",
                               subset_columns = ['House id'])

school_data = remove_duplicates(df = school_data,
                               df_name = "school_data",
                               subset_columns = ['school district'])

population_data = remove_duplicates(df = population_data,
                               df_name = "population_data",
                               subset_columns = ['zip'])

crime_data = remove_duplicates(df = crime_data,
                               df_name = "crime_data",
                               subset_columns = ['zip code'])

## Data formatting

In [ ]:
# replace "k" columns with actual numerical values
pd.options.mode.chained_assignment = None

house_data['House price ($)'] = house_data['House price (k)'] * 1000
house_data.drop(columns=['House price (k)'], inplace=True)

population_data['Population'] = population_data['Population(k)'] * 1000
population_data.drop(columns=['Population(k)'], inplace=True)

house_data[['House id', 'House price ($)']].sample(3)

## Merge and subset the ML dataset

In [ ]:
ml_data = house_data.merge(school_data,
                           how = 'inner',
                           on = 'school district')

ml_data = ml_data.merge(population_data,
                        how = 'inner',
                        left_on = 'zip code',
                        right_on = 'zip')

ml_data = ml_data.merge(crime_data,
                        how = 'inner',
                        on = 'zip code')

print(f'shape of merged ml_data: {ml_data.shape}')
ml_data.sample(5)

# Exploratory Data Analysis (EDA)

## Variable Distributions and Outliers

In [ ]:
# import needed library
import matplotlib.pyplot as plt

# define function to plot histogram and identify outliers
def plot_histogram(df: pd.DataFrame,
                   variable: str,
                   bins=10,
                   color='grey',
                   edgecolor='black',
                   figsize=(7, 2)):

    # set the figure size
    plt.figure(figsize=figsize)

    # plot the histogram
    plt.hist(df[variable],
             bins=bins,
             color=color,
             edgecolor=edgecolor)

    # customize the plot labels and colors
    plt.title(f'{variable} Histogram')
    plt.xlabel(f'{variable}')
    plt.ylabel('Frequency')
    plt.xticks(rotation=45, ha='right')
    plt.ticklabel_format(style='plain', axis='x')
    plt.grid(True)

    # define the Inter Quartile Range (iqr) and outlier bounds
    q1 = df[variable].quantile(0.25)
    q3 = df[variable].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    # mark the outlier boundson the histogram
    plt.axvline(lower_bound, color='blue', linestyle='dashed', linewidth=2, label='Lower Bound')
    plt.axvline(upper_bound, color='blue', linestyle='dashed', linewidth=2, label='Upper Bound')

    # Show the plot
    plt.legend()
    plt.show()

    # count the outliers
    num_outliers = ((df[variable] < lower_bound) | (df[variable] > upper_bound)).sum()

    # print information about outliers
    if num_outliers > 0:
        print(f"{num_outliers} potential outliers detected in {variable} distribution")
        print(f"Lower Bound: {lower_bound}, Upper Bound: {upper_bound}")
    else:
        print(f"no potential outliers detected in {variable} distribution")
        print(f"Lower Bound: {lower_bound}, Upper Bound: {upper_bound}")

    # print a new line
    print("""
          -----
          """)

In [ ]:
# look at histograms for each of your variables
for v in ['size', 'number of bedrooms', 'school rating',
          'Safety Score (0-10)', 'Population', 'House price ($)']:
  plot_histogram(df = ml_data,
                 variable = v,
                 bins = 10)

## Collinearity

In [ ]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

# create a correlation matrix
corr_matrix = ml_data[['size', 'number of bedrooms',
                       'school rating', 'Safety Score (0-10)',
                       'Population']].corr()

# create a heatmap from the corr matrix
sns.set(style="white")
plt.figure(figsize=(7, 3))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix,
            annot=True,
            cmap='coolwarm',
            fmt=".2f",
            linewidths=.5,
            mask=mask,
            vmin=-1, vmax=1)

# customize the output and show the plot
plt.xticks(rotation=45, ha='right')
plt.title('Feature Correlation Heatmap')
plt.show()

## Linear relationships (Pearson correlation coefficient)
Pearson's correlation coefficient is a statistical measure that focuses on quantifying the linear relationship between two continuous variables. Calculated by dividing the covariance of the two variables by the product of their standard deviations, the coefficient ranges from -1 to 1. A value of -1 indicates a perfect negative linear relationship, 1 indicates a perfect positive linear relationship, and 0 indicates no linear relationship. In the context of linear regression, the Pearson coefficient is employed to assess the linear relationships between independent and dependent variables.

In [ ]:
import pandas as pd
from scipy.stats import pearsonr

# create an empty dataframe
correlation_df = pd.DataFrame(columns=['Feature', 'Statistic', 'P-value'])

# loop through each feature to get the pearson coefficient
for feature in ['size', 'number of bedrooms', 'school rating', 'Safety Score (0-10)', 'Population']:
    pearson_result = pearsonr(ml_data[feature], ml_data['House price ($)'])

    # append the result to the dataframe
    correlation_df.loc[len(correlation_df.index)] = [feature,
                                                     round(pearson_result[0], 5),
                                                     round(pearson_result[1], 5)]

# display the dataframe
correlation_df

# Fit your ML Model

## Define your features and target variable

In [ ]:
X = ml_data[['size', 'number of bedrooms', 'school rating',
             'Safety Score (0-10)', 'Population']]
y = ml_data['House price ($)']

## Split your data into training and testing sets

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.25,
                                                    random_state=31)

print(f'size of training data: {X_train.shape}')
print(f'size of testing data: {X_test.shape}')

## Fit your Linear Regression Model

In [ ]:
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score

#create the model
LR_model = linear_model.LinearRegression()

# fit your model on the training data
LR_model.fit(X_train, y_train)

# make predictions using the testing set
y_pred = LR_model.predict(X_test)

# Evaluation your ML Model

## Print Evaluation Metrics
- **R Squared:** The % of the variability in the dependent variable that is explained by the independent variables, ranges from 0-1. 1 is the best.
- **Root Mean Squared Error (RMSE):** the measure of the average magnitude of the errors between predicted and observed values in a regression model. It is calculated by taking the square root of the mean of the squared differences between predicted and actual values.

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

# calculate R squared
r2 = r2_score(y_test, y_pred)
print(f'R squared: {round(r2, 5)}')

# calculate RMSE
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f'Root Mean Squared Error (RMSE): {round(rmse, 2)}')

## Plot Evaluation visualzations

In [ ]:
# scatter plot to show actual value vs the models predicted value
# (y actual vs y predicted)
import matplotlib.pyplot as plt

# plot the points on a scatter plot
plt.scatter(y_test, y_pred,
            alpha=0.2, label='Actual vs. Predicted')

# plot a line that represents perfectly accurate predictions
plt.plot([min(y_test), max(y_test)],
         [min(y_test), max(y_test)],
         color='red', linestyle='--',
         label='Perfect Accuracy')

# customize the plot
plt.title('Actual vs. Predicted Values')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.legend()
plt.show()

# Improve your ML Model
- Try removing features from the models
- Try different ways to draw your regression line, like L1 regression (https://scikit-learn.org/stable/modules/linear_model.html#)

In [ ]:
# improve your model here

# Deliver your ML Model to RealEstateML
- after you're satisfied with your models results, deliver the following to RealEstateML
  - a summary of your model: features used, evaluation metrics, plot of predictions vs actuals
  - a python file to implement your model from beginning to end
  - a CSV that shows your models predictions on the testing dataset
  - some ideas for enhancing the model, like additional data points, other evaluation metrics, different supervised ML algorithms, etc